<h1><b>Tiền xử lí dữ liệu (phần 2)

Trong bài này, ta sẽ tiếp tục thực hành những vấn đề:

    - Data transformation:
        + Kỹ thuật Ordinal Encoding và One-hot encoding
        + Kỹ thuật binning: Uniform, Quantile, Clustered
    - Data augmentation: tăng cường dữ liệu để đề phòng trường hợp dữ liệu mất cân đối hoặc các mẫu / phân lớp quá hiếm gặp

<h1>Data Transformation

<h3>Kĩ thuật Ordinal Encoding

Đầu tiên ta cần import thư viện cần thiết `pandas` với tên tắt là `pd`

In [1]:
### BEGIN SOLUTION
import pandas as pd
### END SOLUTION

Tiếp đến là đọc file csv bằng `pandas` như những bài trước và lưu dataframe vào biến `df`. Khi nộp trên hệ thống, các bạn vui lòng đọc tại đường dẫn `data/heart.csv`.

In [ ]:
### BEGIN SOLUTION
df = pd.read_csv("data/heart.csv")
### END SOLUTION
df.head()

Ta thấy ở cột `ChestPain`, cột `Thal`, cột `AHD` có dữ liệu dạng **category**.
Giả sử mô hình của ta chỉ nhận dữ liệu đầu vào dạng số, ta phải biến các cột này thành dạng số sao cho những đặc trưng này không bị thay đổi

Ở cột `ChestPain`:
*   typical: 1
*   asymptomatic: 2
*   nonanginal: 3
*   nontypical: 4

Ở cột `Thal`:
*   normal: 1
*   fixed: 2
*   reversable: 3

Ở cột `AHD`:
*   No: 0
*   Yes: 1

In [ ]:
### BEGIN SOLUTION

import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Khởi tạo
categories = [['typical', 'asymptomatic', 'nonanginal', 'nontypical'],
              ['normal', 'fixed', 'reversable'],
              ['No', 'Yes']]

# Sử dụng OrdinalEncoder
encoder = OrdinalEncoder(categories=categories)
df[['ChestPain', 'Thal', 'AHD']] = encoder.fit_transform(df[['ChestPain', 'Thal', 'AHD']])
df[['ChestPain', 'Thal']] += 1

# Chuyển đổi kiểu dữ liệu thành int64
df[["ChestPain", "Thal", "AHD"]] = df[["ChestPain", "Thal", "AHD"]].astype("int64")

### END SOLUTION

df.head()

In [4]:
assert df['ChestPain'].dtypes == 'int64'
assert df.shape == (50, 15)

<h3>Kĩ thuật One-hot encoding

Ở đây ta sẽ tiếp tục đi đến một kĩ thuật khác đó là `One-hot`

Ta tiếp tục đọc file csv bằng `pandas` và lưu dataframe vào biến `df`. Khi nộp trên hệ thống, các bạn vui lòng đọc tại đường dẫn `data/heart.csv`.

In [ ]:
### BEGIN SOLUTION
df = pd.read_csv("data/heart.csv")
### END SOLUTION
df.head()

Ở cột `Thal`, ta sẽ biến cột này thành `One-hot` bằng cách thêm 3 cột `fixed`, `normal`, `reversable` vào `df`

**Ba cột mới này sẽ biểu diễn One-hot cho cột `Thal` cũ**

In [ ]:
### BEGIN SOLUTION

# Thực hiện one-hot encoding cho cột 'Thal'
df = pd.get_dummies(df, columns=["Thal"])

# Đổi tên các cột mới thành 'fixed', 'normal', 'reversable'
df.rename(columns={"Thal_fixed": "fixed", "Thal_normal": "normal", "Thal_reversable": "reversable"}, inplace=True)

# Chuyển các giá trị ở các cột thành 0 hoặc 1
df[['fixed', 'normal', 'reversable']] = df[['fixed', 'normal', 'reversable']].astype(int)

### END SOLUTION

df

In [8]:
assert df.shape == (50, 17)
assert df['fixed'][0] == 1
assert df['normal'][0] == 0
assert df['reversable'][0] == 0

<i>Trong một số trường hợp, ta không thể giữ nguyên dữ liệu dạng số để tính toán, mà ta cần chuyển nó qua dạng category

Thường thì các mô hình Tree sẽ cần dữ liệu dạng Category </i>

Để làm được việc này, ta sẽ đi đến kĩ thuật đầu tiên `Uniform`

<h3>Kĩ thuật Uniform

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(df['Age'])
plt.xlabel('Index')
plt.ylabel('Age');

**Kĩ thuật Uniform dùng để biến dữ liệu thành các Bin có khoảng dữ liệu bằng nhau**

Mục tiêu: Tạo thêm **1 cột**, giá trị của cột là `middle-age` hoặc `old` hoặc `elderly`, cột có label là `Uniform`, cột này ở kế sau cột `Age`

Ta có thể làm thủ công hoặc sử dụng phương thức `qcut` của `pandas`

In [11]:
### BEGIN SOLUTION

# Tạo các bin có khoảng dữ liệu bằng nhau cho cột "Age"
bins = pd.qcut(df['Age'], q=3, labels=["middle-age", "old", "elderly"])

# Thêm cột mới vào DataFrame
df.insert(2, 'Uniform', bins)

### END SOLUTION

In [ ]:
import seaborn as sns
sns.countplot(x=df['Uniform']);

Ta có thể thấy số lượng dữ liệu chênh nhau không quá nhiều

In [13]:
assert df.columns[2] == 'Uniform'
assert df['Uniform'].dtype == 'category'

<h3>Kĩ thuật Quantile

Kĩ thuật Quantile dùng để chia dữ liệu thành những khoảng với phần trăm cho trước

Giả sử ta sẽ chia cột `Age` thành 4 phần:
* `Q1` <= 25%
* 25% < `Q2` <= 50%
* 50% < `Q3` <= 95%
* 95% < `Q4`

Sau đó ta thêm cột `Quantile` với các giá trị `Q1, Q2, Q3, Q4` ở vị trí thứ 3 của dataframe `df`

In [15]:
labels = ['Q1', 'Q2', 'Q3', 'Q4']
quantiles = df['Age'].quantile([0.25, 0.5, 0.95])
### BEGIN SOLUTION

bins = pd.qcut(df['Age'], q=[0, 0.25, 0.5, 0.95, 1], labels=labels)

# Thêm cột 'Quantile' vào vị trí thứ 3 của dataframe
df.insert(3, 'Quantile', bins)

### END SOLUTION

In [ ]:
sns.countplot(x=df['Quantile']);

In [17]:
assert df.shape == (50, 19)

<h3>Kĩ thuật Clustered

Kĩ thuật Clustered dùng để biến dữ liệu theo ý của ta

Ở đây là các tuổi lớn hơn **60** là `C1` và còn lại là `C2`

Ta tiếp tục thêm cột `Clustered` ở vị trí thứ `4`, giá trị của cột này là `C1, C2`

In [19]:
labels = ['C1', 'C2']
### BEGIN SOLUTION

# Định nghĩa hàm để phân loại tuổi
def categorize_age(age, labels):
    if age > 60:
        return labels[0]
    else:
        return labels[1]

# Áp dụng hàm cho cột "Age" và tạo cột "Clustered"
bins = df['Age'].apply(categorize_age, args=(labels,))

# Đảm bảo cột "Clustered" được đặt ở vị trí thứ 4
df.insert(4, 'Clustered', bins)

### END SOLUTION

In [ ]:
sns.countplot(x=df['Clustered']);

<h1>Data augmentation

Đối với những bài toán mà quá ít dữ liệu, ta có thể tìm thêm hoặc tạo ra dữ liệu mới từ những dữ liệu hiện có

Nhưng điều này ít nhiều cũng sẽ ảnh hưởng đến chất lượng của mô hình, ta nên cân nhắc kĩ khi sử dụng cách này